In [1]:

%pip install pandas openpyxl beautifulsoup4 requests selenium


Note: you may need to restart the kernel to use updated packages.


In [116]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

import time
from random import randint

class Product:
    def __init__(self, mpn, product_title=None, product_description=None, price=None, dimensions=None, packaging=None, oum=None, manufacturer_name=None):
        self.mpn = mpn
        self.product_title = product_title
        self.product_description = product_description
        self.price = price
        self.dimensions = dimensions
        self.packaging = packaging
        self.oum = oum
        self.manufacturer_name = manufacturer_name

    def __repr__(self):
        return f"Product(MPN={self.mpn}, Product_Title={self.product_title}, Product_Description={self.product_description}, Price={self.price}, Dimensions={self.dimensions}, Packaging={self.packaging}, OUM={self.oum}, Manufacturer_Name={self.manufacturer_name})"
    
def convert_excel_to_csv(input_excel_file, output_csv_file):
    df = pd.read_excel(input_excel_file)
    df.to_csv(output_csv_file, index=False)

def read_mpn_from_csv(file_path):
    products = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        #TODO: Add header logic here if we need
        next(reader)  # Skip header row if present
        for row in reader:
            products.append(Product(mpn=row[0], product_description=row[1], manufacturer_name=row[2]))
    return products

def scrape_price_for_mpn_raw(mpn):
    url = f"https://example.com/search?q={mpn}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        price_element = soup.find('span', class_='price')  # Update with actual structure
        if not price_element:
            print(f"Price element not found for MPN {mpn}")
            return None
        price = price_element.text.strip()
        return price
    except Exception as e:
        print(f"Error scraping {mpn}: {e}")
        return None
    
def scrape_values_for_mpn_ga_international(passed_product):
    """
    Uses Selenium to find the search bar, enter the MPN, and select an item from the search results.
    """
    mpn = str(passed_product.mpn)
    # Replace with the URL of the website you are scraping
    url = "https://www.avantorsciences.com"
    
    # Path to your WebDriver executable
    driver_path = "/chromedriver"  # Replace with the actual path to your WebDriver
    
    # Initialize the WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run browser in headless mode
    options.add_argument('--disable-gpu')  # Disable GPU for headless
    options.add_argument('--no-sandbox')
    
    driver = webdriver.Chrome(options=options)
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
    
    try:
        # Navigate to the website
        driver.get(url+"/us/en/")
        
        # Wait for the search bar to load
        wait = WebDriverWait(driver, timeout=10, poll_frequency=.2,ignored_exceptions=ignored_exceptions)
        search_bar = wait.until(EC.presence_of_element_located((By.TAG_NAME, "input")))  # Replace `name="search"` with the actual locator
        
        # Enter the MPN into the search bar
        search_bar.clear()
        search_bar.send_keys(mpn)
        search_bar.send_keys(Keys.RETURN)
        
        # Wait for the search results to load
        WebDriverWait(driver=driver, timeout=10, ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.CLASS_NAME, "cx-product-search-list")))
        
        
        extension = driver.find_element(By.CLASS_NAME, "cx-product-name").get_attribute('href')  # Replace with the actual class name
        print(extension)
        # s = randint(1,10)
        # time.sleep(s)
        
        # Go to product page
        driver.get(extension)
        # time.sleep(10)
        table_list = []
        #Wait for product page to load
        table_list = wait.until(EC.presence_of_all_elements_located(((By.TAG_NAME, "app-avtr-product-variant-table"))))
        
        # Go through multiple product tables and search for MPN match
        products = []
        for table in table_list:
            table_soup = BeautifulSoup(table.get_attribute('innerHTML'), "html.parser")
            products = table_soup.find_all(class_="item item-container spec-border")
            for product in products:
                supplier_num = str(product.find_all(title="Supplier #")[0].text).replace(" ","")
                if supplier_num == mpn:
                    print('FOUND: \n' + mpn)
                    process_product_ga_international(product, passed_product)
                    print(passed_product)
                    return

    except Exception as e:
        print(f"Error with MPN {mpn}: {e}")
        return None
    finally:
        # Close the browser
        driver.quit()    
    
def scrape_products(products):
    for product in products:
        if product.manufacturer_name == "GA INTERNATIONAL INC":
            scrape_values_for_mpn_ga_international(product)
        elif product.manufacturer_name == "Brady Worldwide":
            print("Brady Worldwide")
            # product = scrape_values_for_mpn_brady_worldwide(product.mpn)
        else:
            print("ERROR: Manufacturer Name not recognized")

def write_to_csv(products, output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["MPN", "Product Title", "Product Description", "Price", "Dimensions", "Packaging", "OUM", "Manufacturer Name"])
        for product in products:
            writer.writerow([product.mpn, product.product_title, product.product_description, product.price, product.dimensions, product.packaging, product.oum, product.manufacturer_name])

# self.mpn = mpn
#         self.product_title = product_title
#         self.product_description = product_description
#         self.price = price
#         self.dimensions = dimensions
#         self.packaging = packaging
#         self.oum = oum
#         self.manufacturer_name = manufacturer_name            

def write_to_excel(products, output_file):
    data = [{"MPN": product.mpn, "Product Title": product.product_title, "Product Description": product.product_description, "Price": product.price, "Dimensions": product.dimensions, "Packaging": product.packaging, "OUM": product.oum, "Manufacturer Name": product.manufacturer_name} for product in products]
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)

def process_product_ga_international(product, passed_product):
    price = str(product.find_all(title="Price")[0].find_all(class_="font-bold desktop-only")[0].text).replace(" ","")
    # product_title = str(product.find_all(title="Price")[0].text).replace(" ","")
    dimensions = str(product.find_all(title="Dimensions", class_="attribute")[0].text).replace(" ","")
    packaging = str(product.find_all(title="Packaging", class_="attribute")[0].text).replace(" ","")
    passed_product.dimensions = dimensions
    passed_product.packaging = packaging
    passed_product.price = price
    return



In [104]:

temp = "807656001548"
soup = scrape_price_for_mpn_ga_international(temp)    

https://www.avantorsciences.com/us/en/product/28504773/deep-freeze-label
FOUND: 
807656001548
Product(MPN=807656001548, Product_Title=None, Price=$119.32, Dimensions=76.2×25.4mm(3×1"), Packaging=1000labels/roll(1labelacross), OUM=None)


In [118]:

# # Define input and output file paths
input_excel_file = "Brady and GA International copy.xlsx"  # Replace with your input Excel file
temp_csv_file = "mpns.csv"
output_csv_file = "prices.csv"
output_excel_file = "output.xlsx"

# # Step 1: Convert Excel to CSV
convert_excel_to_csv(input_excel_file, temp_csv_file)
print("Excel converted to CSV.")

# # Step 2: Read MPNs from CSV
products = read_mpn_from_csv(temp_csv_file)
print("Products read from CSV:", products)

# # Step 3: Scrape Prices
scrape_products(products)
print("Prices scraped:", products)

# # Step 4: Write to CSV
write_to_csv(products, output_csv_file)
print(f"Prices written to {output_csv_file}")

# # Step 5: Write to Excel
write_to_excel(products, output_excel_file)
print(f"Prices written to {output_excel_file}")


Excel converted to CSV.
Products read from CSV: [Product(MPN=807656001548, Product_Title=None, Product_Description=Deep-Freeze Labels for Thermal-Transfer Barcode Printers, FreezerTAG, GA International - 807656001548, Cryo labels for thermal-transfer barcode printers, Price=None, Dimensions=None, Packaging=None, OUM=None, Manufacturer_Name=GA INTERNATIONAL INC), Product(MPN=807656001555, Product_Title=None, Product_Description=Deep-Freeze Labels for Thermal-Transfer Barcode Printers, FreezerTAG, GA International - 807656001555, Cryo labels for thermal-transfer barcode printers, Price=None, Dimensions=None, Packaging=None, OUM=None, Manufacturer_Name=GA INTERNATIONAL INC), Product(MPN=807656001562, Product_Title=None, Product_Description=Deep-Freeze Labels for Thermal-Transfer Barcode Printers, FreezerTAG, GA International - 807656001562, Cryo labels for thermal-transfer barcode printers, Price=None, Dimensions=None, Packaging=None, OUM=None, Manufacturer_Name=GA INTERNATIONAL INC), Pro